# Initialization

In [ ]:
from econia.account import Account
from econia.rest import EconiaClient, move_trio, typed_trio
import econia.defs as defs

In [ ]:
client = EconiaClient()

## Users

In [ ]:
econia = Account(dev_nb=True)
user = Account()

In [ ]:
# Aliases
vn_url = client.tx_vn_url_print
coins_module = defs.econia_modules.Coins.name
bct_struct = defs.econia_modules.Coins.structs.BCT
qct_struct = defs.econia_modules.Coins.structs.QCT
BCT = move_trio(econia.address(), coins_module, bct_struct)
QCT = move_trio(econia.address(), coins_module, qct_struct)
CoinStore = move_trio(
    defs.named_addrs.Std,
    defs.module_names.Coin,
    defs.member_names.CoinStore)
bct_coinstore = typed_trio(CoinStore, BCT)
qct_coinstore = typed_trio(CoinStore, QCT)
E1_struct = defs.econia_modules.Registry.structs.E1
registry_module = defs.econia_modules.Registry.name
register_market_func = defs.econia_modules.Registry.\
    script_functions.register_market
E1 = move_trio(econia.address(), registry_module, E1_struct)
user_mod = defs.econia_modules.User.name
init_user_func = defs.econia_modules.User.\
    script_functions.init_user
init_containers_func = defs.econia_modules.User.\
    script_functions.init_containers
deposit_func = defs.econia_modules.User.\
    script_functions.deposit
submit_ask_func = defs.econia_modules.User.\
    script_functions.submit_ask
match_mod = defs.econia_modules.Match.name
submit_market_buy_func = defs.econia_modules.Match.\
    script_functions.submit_market_buy

In [ ]:
vn_url(client.mint_testcoin(user.address()))

## Econia core resources

Send initialization trigger

In [ ]:
vn_url(client.init_econia(econia))

Verify core structs initialized

In [ ]:
client.core_structs(econia.address())

## Coin types

Initialize mock coin types

In [ ]:
vn_url(client.init_coin_types(econia))

Register a coin store for Econia and for test user

In [ ]:
vn_url(client.register_coin_store(econia, BCT))
vn_url(client.register_coin_store(econia, QCT))
vn_url(client.register_coin_store(user, BCT))
vn_url(client.register_coin_store(user, QCT))

Mint BCT and QCT to each party

In [ ]:
vn_url(client.coins_mint_to(econia, econia.address(), 1_000_000, 2_000_000))
vn_url(client.coins_mint_to(econia, user.address(), 3_000_000, 4_000_000))

Verify holdings for each party

In [ ]:
print(client.account_resource(econia.address(), bct_coinstore))
print(client.account_resource(econia.address(), qct_coinstore))
print(client.account_resource(user.address(), bct_coinstore))
print(client.account_resource(user.address(), qct_coinstore))

# Market/user registration 

Register a test market with scale exponent 1

In [ ]:
vn_url(client.run_script(
    econia,
    [econia.address(), registry_module, register_market_func],
    type_args=[BCT, QCT, E1]
))

Init each user's sequence counter

In [ ]:
vn_url(client.run_script(
    econia,
    [econia.address(), user_mod, init_user_func],
))
vn_url(client.run_script(
    user,
    [econia.address(), user_mod, init_user_func],
))

Init each user with an order collateral container for the test market

In [ ]:
vn_url(client.run_script(
    econia,
    [econia.address(), user_mod, init_containers_func],
    type_args=[BCT, QCT, E1]
))
vn_url(client.run_script(
    user,
    [econia.address(), user_mod, init_containers_func],
    type_args=[BCT, QCT, E1]
))

Deposit base and quote coins into each user's collateral container

In [ ]:
vn_url(client.run_script(
    econia,
    [econia.address(), user_mod, deposit_func],
    [str(500_000), str(600_000)],
    type_args=[BCT, QCT, E1]
))
vn_url(client.run_script(
    user,
    [econia.address(), user_mod, deposit_func],
    [str(700_000), str(800_000)],
    type_args=[BCT, QCT, E1]
))

# Filling against the book 

Have Econia account submit a series of asks

In [ ]:
vn_url(client.run_script(
    econia,
    [econia.address(), user_mod, submit_ask_func],
    [econia.address(), str(20), str(7)],
    type_args=[BCT, QCT, E1]
))
vn_url(client.run_script(
    econia,
    [econia.address(), user_mod, submit_ask_func],
    [econia.address(), str(21), str(15)],
    type_args=[BCT, QCT, E1]
))
vn_url(client.run_script(
    econia,
    [econia.address(), user_mod, submit_ask_func],
    [econia.address(), str(22), str(20)],
    type_args=[BCT, QCT, E1]
))
vn_url(client.run_script(
    econia,
    [econia.address(), user_mod, submit_ask_func],
    [econia.address(), str(23), str(25)],
    type_args=[BCT, QCT, E1]
))
vn_url(client.run_script(
    econia,
    [econia.address(), user_mod, submit_ask_func],
    [econia.address(), str(24), str(30)],
    type_args=[BCT, QCT, E1]
))

Have user account submit a market fill that clears out the book

In [ ]:
vn_url(client.run_script(
    user,
    [econia.address(), match_mod, submit_market_buy_func],
    [econia.address(), str(200), str(400_000)],
    type_args=[BCT, QCT, E1]
))